In [1]:
import json
import pickle
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]

In [3]:
'''Private'''
with open('./Predict/all_prediction_0511SBDA50e0519_epoch16_BM25Fv3ALL_r_0530.pickle', 'rb') as f:
    predict = pickle.load(f)
submit = pd.read_json(path_or_buf='./Private/public_private_submission_template.jsonl', lines=True)
evidence = load_json('./Predict/semanticRes_0511SBDA50e0519_BM25Fv3ALL_0530.jsonl')
evidence = evidence[11620:]

In [4]:
predict = [int(l) for l in list(predict)]

label = []
for i in predict: 
    if i == 0:
        label.append('supports')
    if i == 1:
        label.append('refutes')
    if i == 2:
        label.append('NOT ENOUGH INFO')

In [5]:
evidence[836] = [['', [259, '', ]]]
evidence[836]

[['None', [259, 'None', 'None']]]

In [6]:
evl = {}
num = 0
for i in tqdm(range(len(evidence))):
    tem = []
    for ev in evidence[i]:
        tem.append(ev[1][1:])
        idd = ev[1][0]
    evl[idd] = (label[i], tem)

100%|███████████████████████████████████| 9038/9038 [00:00<00:00, 112730.19it/s]


In [7]:
data = [value for key, value in sorted(evl.items())]

In [8]:
ls = [l for l, e in data]
es = [e for l, e in data]

In [9]:
submit['predicted_label'] = ls
submit['predicted_evidence'] = es

In [10]:
for i in range(len(submit)):
    if submit['predicted_label'][i] == 'NOT ENOUGH INFO':
        submit['predicted_evidence'][i] = None
    if submit['predicted_evidence'][i] == [["None", "None"]]:
        submit['predicted_label'][i] = 'NOT ENOUGH INFO'
        submit['predicted_evidence'][i] = None

In [11]:
data = submit.to_dict('index')

In [12]:
data = [x for x in data.values()]

In [13]:
with open(f"./Private/Predict_data.jsonl", "w", encoding="utf8") as f:
    for i in data:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")